In [30]:
#Install kaggle Library
! pip install kaggle

Upload your Kaggle.json file


In [31]:
#Configuring the path of Kaggle.json file
! mkdir -p ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


importing Twitter Sentiment Dataset

In [32]:
# API to fetch the dataset from kaggle
!kaggle datasets download -d kazanova/sentiment140


Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
sentiment140.zip: Skipping, found more recently modified local copy (use --force to force download)


In [33]:
# Extracting the Compressed Dataset

from zipfile import ZipFile
dataset = '/content/sentiment140.zip'

with ZipFile(dataset,'r') as zip:
  zip.extractall()
  print('The dataset is extracted')

The dataset is extracted


importing The Dependencies

In [34]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


In [35]:
import nltk
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [36]:
# Printing the stopwords in English
print(stopwords.words('english'))

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

In [37]:
# Loading the data from csv file to pandas dataframe
twitter_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv',encoding='latin-1')

In [38]:
# Checking gthe the number of rows and columns
twitter_data.shape

(1599999, 6)

In [39]:
# Printing the first 5 rows of the dataframe
twitter_data.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [40]:
# Naming the coloumn and reading the dataset again

coloumn_name = ['target','ids','date','flag','user','text']
twitter_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv', names = coloumn_name, encoding='latin-1')

In [41]:
# Checking gthe number of rows and columns
twitter_data.shape

(1600000, 6)

In [42]:
#printing the first 5 rows of dataframe
twitter_data.head()

,target,ids,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [43]:
# Counting the number  of missing values in the dataset
twitter_data.isnull().sum()

,0
target,0
ids,0
date,0
flag,0
user,0
text,0


In [44]:
# Checking the  Distribution of the target column
twitter_data['target'].value_counts()

,count
target,
0,800000
4,800000


Convert the target  "4" to "1"

In [45]:
twitter_data.replace({'target':{4:1}},inplace=True)

In [46]:
# Checking the istribution of target column
twitter_data['target'].value_counts()

,count
target,
0,800000
1,800000


0 --> Nehative Tweet
1 --> Positive tweet

Stemming

Stemming is the process of reducing a word to its Root word

example : actor,actress,actring = act

In [47]:
port_stem = PorterStemmer()

In [48]:
def stemming(content):
  stemmed_content = re.sub('[^a-zA-Z]',' ',content)
  stemmed_content = stemmed_content.lower()
  stemmed_content = stemmed_content.split()
  stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content = ' '.join(stemmed_content)
  return stemmed_content


In [49]:
twitter_data['stemmed_content'] = twitter_data['text'].apply(stemming)

In [50]:
twitter_data.head()

,target,ids,date,flag,user,text,stemmed_content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot http twitpic com zl awww bummer sho...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset updat facebook text might cri result sch...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichan dive mani time ball manag save rest g...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass behav mad see


In [51]:
print(twitter_data['stemmed_content'])

0          switchfoot http twitpic com zl awww bummer sho...
1          upset updat facebook text might cri result sch...
2          kenichan dive mani time ball manag save rest g...
3                            whole bodi feel itchi like fire
4                              nationwideclass behav mad see
                                 ...                        
1599995                           woke school best feel ever
1599996    thewdb com cool hear old walt interview http b...
1599997                         readi mojo makeov ask detail
1599998    happi th birthday boo alll time tupac amaru sh...
1599999    happi charitytuesday thenspcc sparkschar speak...
Name: stemmed_content, Length: 1600000, dtype: object


In [52]:
print(twitter_data['target'])

0          0
1          0
2          0
3          0
4          0
          ..
1599995    1
1599996    1
1599997    1
1599998    1
1599999    1
Name: target, Length: 1600000, dtype: int64


In [53]:
#seprating the data and label
x = twitter_data['stemmed_content'].values
y = twitter_data['target'].values

In [54]:
print(x)

['switchfoot http twitpic com zl awww bummer shoulda got david carr third day'
 'upset updat facebook text might cri result school today also blah'
 'kenichan dive mani time ball manag save rest go bound' ...
 'readi mojo makeov ask detail'
 'happi th birthday boo alll time tupac amaru shakur'
 'happi charitytuesday thenspcc sparkschar speakinguph h']


In [55]:
print(y)

[0 0 0 ... 1 1 1]


Splitting the data to training and test data

In [56]:
X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size = 0.2, stratify=y, random_state=2)

In [57]:
print(x.shape, X_train.shape, X_test.shape)

(1600000,) (1280000,) (320000,)


In [59]:
print(X_train)

['watch saw iv drink lil wine' 'hatermagazin'
 'even though favourit drink think vodka coke wipe mind time think im gonna find new drink'
 ... 'eager monday afternoon'
 'hope everyon mother great day wait hear guy store tomorrow'
 'love wake folger bad voic deeper']


In [60]:
print(X_test)

['mmangen fine much time chat twitter hubbi back summer amp tend domin free time'
 'ah may show w ruth kim amp geoffrey sanhueza'
 'ishatara mayb bay area thang dammit' ...
 'destini nevertheless hooray member wonder safe trip' 'feel well'
 'supersandro thank']


In [61]:
# Coverting  the textual data to numerical data
vectorizer = TfidfVectorizer()
vectorizer.fit(x)

X_train = vectorizer.transform(X_train)
X_test = vectorizer.transform(X_test)

In [62]:
print(X_train)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 9453092 stored elements and shape (1280000, 532699)>
  Coords	Values
  (0, 126118)	0.37484032460318173
  (0, 213554)	0.5289928671316323
  (0, 271144)	0.42037613603065954
  (0, 409438)	0.35861719248416485
  (0, 504044)	0.2725664180116586
  (0, 511398)	0.4472655127083222
  (1, 185304)	1.0
  (2, 90030)	0.31341180319788786
  (2, 126118)	0.45825085123736603
  (2, 143635)	0.18906085394224742
  (2, 149331)	0.29041017783193285
  (2, 152669)	0.20261379420870737
  (2, 173898)	0.1880703874098935
  (2, 205312)	0.16175394132247145
  (2, 307827)	0.2410455646561603
  (2, 333015)	0.16780678648815192
  (2, 468994)	0.32099337313155857
  (2, 470034)	0.18720443014780438
  (2, 472107)	0.15171423197968206
  (2, 500375)	0.3297483694639193
  (2, 511810)	0.33617620658621283
  (3, 65151)	0.5205345321237522
  (3, 175126)	0.27847482359309217
  (3, 183099)	0.44549570924150195
  (3, 198842)	0.3743117172596608
  :	:
  (1279996, 336028)	0.17877262371579808

Training The Machine Learning Model


Logistic Regression

In [63]:
model = LogisticRegression(max_iter=1000)

In [66]:
model.fit(X_train,Y_train)

LogisticRegression(max_iter=1000)

Model Evalution

Accuracy score

In [69]:
# Accuracy score on the training data
x_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(Y_train, x_train_prediction)

In [70]:
print('Accuracy score on the training data : ',training_data_accuracy)

Accuracy score on the training data :  0.8047734375


In [71]:
# Accuracy score on the test data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(Y_test, X_test_prediction)

In [72]:
print('Accuracy score on the test data:', test_data_accuracy)

Accuracy score on the test data: 0.777475


Model Accuracy = 77.8 %

In [73]:
import pickle

In [74]:
filename = 'trained_model.sav'
pickle.dump(model,open(filename,'wb'))

Using the saved model for future model

In [78]:
loaded_model = pickle.load(open('/content/trained_model.sav','rb'))

x_new = X_test[200]
print(Y_test[200])

prediction = model.predict(x_new)
print(prediction)
if (prediction[0] == 0):
  print('Negative Tweet')
else:
    print('Positive Tweet')

1
[1]
Positive Tweet


In [80]:
loaded_model = pickle.load(open('/content/trained_model.sav','rb'))

x_new = X_test[3]
print(Y_test[3])

prediction = model.predict(x_new)
print(prediction)
if (prediction[0] == 0):
  print('Negative Tweet')
else:
    print('Positive Tweet')

0
[0]
Negative Tweet
